In [6]:
import numpy as np
import librosa
from pedalboard import Pedalboard, Distortion, Reverb, Delay, Chorus

# Parameters
sample_rate = 22050
n_mels = 128
frame_length = 2048
hop_length = 512

# Define effect plugins to use
effects_list = [
    ("distortion", Distortion(drive_db=0.8)),   # example parameter
    ("reverb", Reverb(room_size=0.5)),
    ("delay", Delay(delay_seconds=0.3)),
    ("chorus", Chorus(depth=0.5))
]
# We can also include combinations explicitly if desired:
combo_effects_list = [
    ("distortion_reverb", [Distortion(drive_db=0.8), Reverb(room_size=0.5)]),
    ("distortion_delay",  [Distortion(drive_db=0.8), Delay(delay_seconds=0.3)])
]
# The above combos show two at a time. We could add more combinations similarly.


In [26]:
from os import listdir
from os.path import join

# Path to GuitarSet files
guitarset_path = "data/guitarset"
all_clean_guitar_files = [join(guitarset_path, f) for f in listdir(guitarset_path)]
# print(clean_guitar_files)
print(len(all_clean_guitar_files))

# Use first 100 for now
clean_guitar_files = all_clean_guitar_files[:360]
print(clean_guitar_files)

360
['data/guitarset/03_Rock2-85-F_comp_mix.wav', 'data/guitarset/01_SS2-107-Ab_comp_mix.wav', 'data/guitarset/05_Rock3-148-C_solo_mix.wav', 'data/guitarset/04_SS1-100-C#_solo_mix.wav', 'data/guitarset/05_Jazz3-137-Eb_solo_mix.wav', 'data/guitarset/00_BN3-154-E_comp_mix.wav', 'data/guitarset/05_Rock1-130-A_solo_mix.wav', 'data/guitarset/05_Funk1-114-Ab_solo_mix.wav', 'data/guitarset/00_BN2-166-Ab_solo_mix.wav', 'data/guitarset/04_Funk3-112-C#_comp_mix.wav', 'data/guitarset/01_BN2-166-Ab_comp_mix.wav', 'data/guitarset/02_Rock3-148-C_comp_mix.wav', 'data/guitarset/04_SS2-88-F_solo_mix.wav', 'data/guitarset/02_SS3-84-Bb_solo_mix.wav', 'data/guitarset/01_Rock3-148-C_solo_mix.wav', 'data/guitarset/05_SS1-100-C#_comp_mix.wav', 'data/guitarset/02_Rock1-130-A_comp_mix.wav', 'data/guitarset/04_Jazz2-187-F#_solo_mix.wav', 'data/guitarset/02_Rock2-85-F_solo_mix.wav', 'data/guitarset/04_Rock3-117-Bb_comp_mix.wav', 'data/guitarset/00_SS2-107-Ab_solo_mix.wav', 'data/guitarset/01_Rock1-130-A_solo_mix

In [27]:
import os

# Directory to save processed audio files
output_dir = "data/generated"
os.makedirs(output_dir, exist_ok=True)

# Create a list to store metadata
metadata = []

In [28]:
from tqdm import tqdm
import soundfile as sf

# Total iterations (clean files * (single effects + combo effects))
total_iterations = len(clean_guitar_files) * (len(effects_list) + len(combo_effects_list))

# Progress bar setup
progress_bar = tqdm(total=total_iterations, desc="Processing Audio", unit="file")
# Iterate over clean guitar audio files
for clean_path in clean_guitar_files:  
    # Load clean audio
    y, sr = librosa.load(clean_path, sr=sample_rate)
    
    # Get the base filename without extension
    base_filename = os.path.splitext(os.path.basename(clean_path))[0]
    
    # Compute mel spectrogram for clean audio
    S_clean = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, 
                                         n_fft=frame_length, hop_length=hop_length)

    S_clean_db = librosa.power_to_db(S_clean, ref=np.max)
    
    # Normalize spectrogram
    S_clean_db = (S_clean_db - S_clean_db.min()) / (S_clean_db.max() - S_clean_db.min() + 1e-6)
    S_clean_db = S_clean_db[..., np.newaxis]

    # Apply each single effect and save processed audio
    for effect_name, effect_plugin in effects_list:
        board = Pedalboard([effect_plugin])
        processed_audio = board(y, sample_rate=sr)
        
        # Save processed audio file
        effect_filename = f"{base_filename}_{effect_name}.wav"
        effect_filepath = os.path.join(output_dir, effect_filename)
        sf.write(effect_filepath, processed_audio, sr)
        
        # Compute mel spectrogram of processed audio
        S_proc = librosa.feature.melspectrogram(y=processed_audio, sr=sr, 
                                                n_mels=n_mels, n_fft=frame_length, hop_length=hop_length)
        S_proc_db = librosa.power_to_db(S_proc, ref=np.max)
        S_proc_db = (S_proc_db - S_proc_db.min()) / (S_proc_db.max() - S_proc_db.min() + 1e-6)
        S_proc_db = S_proc_db[..., np.newaxis]

        # Save metadata
        metadata.append({"clean_file": clean_path, "processed_file": effect_filepath, "effect": effect_name})
        
        # Update progress bar
        progress_bar.update(1)
    
    # Apply effect combinations and save processed audio
    for combo_name, plugin_list in combo_effects_list:
        board = Pedalboard(plugin_list)
        processed_audio = board(y, sample_rate=sr)
        
        # Save processed audio file
        combo_filename = f"{base_filename}_{combo_name}.wav"
        combo_filepath = os.path.join(output_dir, combo_filename)
        sf.write(combo_filepath, processed_audio, sr)
        
        # Compute mel spectrogram
        S_proc = librosa.feature.melspectrogram(y=processed_audio, sr=sr, 
                                                n_mels=n_mels, n_fft=frame_length, hop_length=hop_length)
        S_proc_db = librosa.power_to_db(S_proc, ref=np.max)
        S_proc_db = (S_proc_db - S_proc_db.min()) / (S_proc_db.max() - S_proc_db.min() + 1e-6)
        S_proc_db = S_proc_db[..., np.newaxis]

        # Save metadata
        metadata.append({"clean_file": clean_path, "processed_file": combo_filepath, "effect": combo_name})
        
        # Update progress bar
        progress_bar.update(1)

# Close progress bar
progress_bar.close()

Processing Audio: 100%|██████████| 2160/2160 [01:00<00:00, 35.77file/s]


In [29]:
import pandas as pd

# Save metadata to a CSV file
metadata_df = pd.DataFrame(metadata)
metadata_df.to_csv(os.path.join(output_dir, "processed_audio_metadata.csv"), index=False)

print("Processed audio files saved. Metadata stored in 'processed_audio_metadata.csv'.")

Processed audio files saved. Metadata stored in 'processed_audio_metadata.csv'.


In [11]:
# # Prepare training data
# X_clean_specs = []
# X_proc_specs  = []
# Y_labels = []

# X_clean_specs = 

# # Convert lists to numpy arrays for training
# X_clean_specs = np.array(X_clean_specs)
# X_proc_specs  = np.array(X_proc_specs)
# Y_labels = np.array(Y_labels)

# # Split into train/val/test
# from sklearn.model_selection import train_test_split
# Xc_train, Xc_val, Xp_train, Xp_val, y_train, y_val = train_test_split(
#     X_clean_specs, X_proc_specs, Y_labels, test_size=0.2, random_state=42)

# # Train the model
# history = model.fit(
#     {"clean_input": Xc_train, "processed_input": Xp_train}, y_train,
#     validation_data=({"clean_input": Xc_val, "processed_input": Xp_val}, y_val),
#     epochs=20, batch_size=16
# )

# # After training, evaluate on a test set (not shown: you'd similarly create Xc_test, Xp_test, y_test)
# test_loss, test_acc = model.evaluate({"clean_input": Xc_test, "processed_input": Xp_test}, y_test)
# print("Test accuracy:", test_acc)


KeyboardInterrupt: 